In [ ]:
!pip install torch transformers sentence-transformers numpy nltk spacy

In [ ]:
import json
import spacy
from collections import deque
from transformers import BlenderbotTokenizer, BlenderbotForConditionalGeneration

# Cargar el modelo de Blenderbot
model_name = "facebook/blenderbot-400M-distill"
tokenizer = BlenderbotTokenizer.from_pretrained(model_name)
model = BlenderbotForConditionalGeneration.from_pretrained(model_name)

# Cargar intents
with open("intents.json", "r", encoding="utf-8") as file:
    intents = json.load(file)

# Cargar datos de celulares
with open("celulares_limpios.json", "r", encoding="utf-8") as file:
    celulares = json.load(file)

# Memoria del chatbot
memoria_chat = deque(maxlen=5)
preferencias_usuario = {}
inicio_recoleccion = False
preguntas_flujo = [
    {"clave": "bateria", "pregunta": "¿Te interesa que tenga una batería de larga duración? (Sí/No)"},
    {"clave": "presupuesto", "pregunta": "¿Cuánto es tu presupuesto máximo? (Ingresa un número)"},
    {"clave": "marca", "pregunta": "¿Qué marca prefieres? (Escribe una marca específica)"},
    {"clave": "camara", "pregunta": "¿Quieres una buena cámara? (Sí/No)"},
    {"clave": "rendimiento", "pregunta": "¿Necesitas un celular con buen rendimiento? (Sí/No)"}
]
indice_pregunta = 0

# Cargar modelo de lenguaje para extraer palabras clave
nlp = spacy.load("es_core_news_sm")

def extraer_palabras_clave(texto):
    doc = nlp(texto)
    palabras_clave = [token.lemma_ for token in doc if token.pos_ in ["NOUN", "ADJ", "VERB", "NUM"]]
    return palabras_clave

def recomendar_celular():
    celulares_filtrados = celulares
    for clave, valor in preferencias_usuario.items():
        if clave == "presupuesto":
            celulares_filtrados = [c for c in celulares_filtrados if c["Launched Price (USA)"] <= valor]
        elif clave == "camara":
            celulares_filtrados = [c for c in celulares_filtrados if c["Front Camera (MP)"] >= valor]
        elif clave == "rendimiento":
            celulares_filtrados = [c for c in celulares_filtrados if c["RAM (GB)"] >= valor]
        elif clave == "bateria":
            celulares_filtrados = [c for c in celulares_filtrados if c["Battery Capacity (mAh)"] >= valor]
        elif clave == "marca":
            celulares_filtrados = [c for c in celulares_filtrados if valor.lower() in c["Company Name"].lower()]
    
    return celulares_filtrados[0] if celulares_filtrados else None

def generar_mensaje_recomendacion(celular):
    mensaje = f"Te recomiendo el {celular['Model Name']} de {celular['Company Name']}."

    if "presupuesto" in preferencias_usuario:
        mensaje += f" Su precio de lanzamiento es ${celular['Launched Price (USA)']}."

    if "bateria" in preferencias_usuario and preferencias_usuario["bateria"] > 0:
        mensaje += f" Tiene una batería de {celular['Battery Capacity (mAh)']} mAh para una excelente duración."

    if "camara" in preferencias_usuario and preferencias_usuario["camara"] > 0:
        mensaje += f" Su cámara frontal es de {celular['Front Camera (MP)']} MP y la trasera de {celular['Back Camera (MP)']} MP."

    if "rendimiento" in preferencias_usuario and preferencias_usuario["rendimiento"] > 0:
        mensaje += f" Ofrece un gran rendimiento gracias a su procesador {celular['Processor']} y {celular['RAM (GB)']} GB de RAM."

    mensaje += f" Tiene una pantalla de {celular['Screen Size (inches)']} pulgadas."

    return mensaje

def chatbot():
    global inicio_recoleccion, preferencias_usuario, indice_pregunta
    print("Hola, ¿en qué puedo ayudarte?")
    while True:
        user_input = input("Tú: ").strip()

        if not user_input:
            print("Chatbot: No entendí tu respuesta. Por favor, escribe algo.")
            continue

        if user_input.lower() == "salir":
            print("Chatbot: ¡Hasta luego!")
            break

        if not inicio_recoleccion:
            print("Chatbot: ¿Quieres una recomendación de celular? Si es así, dime tus preferencias.")
            inicio_recoleccion = True
            indice_pregunta = 0
            preferencias_usuario.clear()
            print(f"Chatbot: {preguntas_flujo[indice_pregunta]['pregunta']}")
            continue

        while True:
            user_input = input("Tú: ").strip()
            if not user_input:
                print("Chatbot: No entendí tu respuesta. Por favor, escribe algo.")
                continue

            clave_pregunta_actual = preguntas_flujo[indice_pregunta]["clave"]

            if clave_pregunta_actual in ["bateria", "camara", "rendimiento"]:
                if "sí" in user_input.lower():
                    preferencias_usuario[clave_pregunta_actual] = 4000 if clave_pregunta_actual == "bateria" else 1
                elif "no" in user_input.lower():
                    preferencias_usuario[clave_pregunta_actual] = 0
                else:
                    print("Chatbot: Responde con 'Sí' o 'No'.")
                    continue

            elif clave_pregunta_actual == "presupuesto":
                palabras_clave = extraer_palabras_clave(user_input)
                presupuesto_valido = None
                for palabra in palabras_clave:
                    if palabra.isdigit():
                        presupuesto_valido = int(palabra)
                        break
                if presupuesto_valido is not None:
                    preferencias_usuario[clave_pregunta_actual] = presupuesto_valido
                else:
                    print("Chatbot: Por favor, ingresa un valor numérico para el presupuesto.")
                    continue

            elif clave_pregunta_actual == "marca":
                if len(user_input) > 1:
                    preferencias_usuario[clave_pregunta_actual] = user_input
                else:
                    print("Chatbot: Por favor, menciona una marca válida.")
                    continue

            indice_pregunta += 1
            break

        if indice_pregunta < len(preguntas_flujo):
            print(f"Chatbot: {preguntas_flujo[indice_pregunta]['pregunta']}")
        else:
            celular_recomendado = recomendar_celular()
            if celular_recomendado:
                respuesta = generar_mensaje_recomendacion(celular_recomendado)
            else:
                respuesta = "No encontré un celular que cumpla con todas tus preferencias."
            print(f"Chatbot: {respuesta}")

            inicio_recoleccion = False
            preferencias_usuario.clear()
            indice_pregunta = 0

chatbot()


Hola, ¿en qué puedo ayudarte?
Chatbot: ¡Hola! ¿Cómo puedo ayudarte?
Chatbot: ¡Hola! ¿Cómo puedo ayudarte?
Chatbot: ¡Hola! ¿Cómo puedo ayudarte?
Chatbot:  Have you ever been to Buen Duel? It's a Mexican soccer tournament.
Chatbot:  Have you ever been to Buen Diego? It's an island in the Caribbean Sea.
Chatbot:  Have you ever been to Buenos Divide? It's a city in Las Vegas, Nevada.
Chatbot:  Have you ever been to Buenos Divide? It's a city in Las Vegas, Nevada.
Chatbot: ¡Hola! ¿Cómo puedo ayudarte?
Chatbot: ¡Entendido! Dime tus preferencias y te recomendaré un celular.
Chatbot: ¿Te interesa que tenga una batería de larga duración?
Chatbot: ¿Cuánto es tu presupuesto máximo?
Chatbot: ¿Qué marca prefieres?
Chatbot: ¿Quieres una buena cámara?
Chatbot: ¿Necesitas un celular con buen rendimiento?


KeyError: 'precio'